# Regression task - Bike sharing 2

In [1]:
import pandas as pd
data = pd.read_csv('../data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Add some features from the past

Since we have time stamp of every measurement, we can see the data as a time series and use data from the past. Add one or more feature columns computed from the data of the previous hour.

You can use the following pandas methods:
* [df.sort_values('column_name')](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) - sorts the rows of a data frame by the column with name 'column_name'.
* [df.shift(periods)](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shift.html) - Shifts index of a data frame by desired number of periods.

In [2]:
data.sort_values(['dteday', 'hr'])
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


### Data preparation

Prepare train and test data sets.

In [3]:
from sklearn.model_selection import train_test_split

X_all = data[['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']


X_train, X_test, y_train, y_test = train_test_split(
    X_all, 
    y_all,
    random_state=1,
    test_size=0.2)

print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

Train size: 13902
Test size: 3476


### Training a regressor

Train a regressor using the following models:
* [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
* [Support Vector Machines for regression](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) (experiment with different kernels)
* [Gradient Boosted Trees](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (Experiment with different depths and number of trees)

In [4]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

regr = LinearRegression()

#regr = Pipeline([('std', StandardScaler()),
#                 ('svr', svm.SVR(kernel='linear'))])

#regr = GradientBoostingRegressor(n_estimators=100, max_depth=4, loss='lad')

regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values. Decide which model performs best on the given problem.

In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = regr.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 190.1343498273878, std: 181.50427424823098
Test Root mean squared error: 95.19
Test Mean absolute error: 64.88


In [6]:
y_pred = regr.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 95.56
Train Mean absolute error: 64.73


### Feature importance

Print coefficients of the linear regression model and decide which features are most important.


In [7]:
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [  4.46007652  17.99950304   0.21864978  -0.0987548   -4.34903078
   0.45547491   2.693172    -1.61408043  50.27017287  22.48126352
 -60.18523272  19.80733569   0.77493273]
